### Example: Non-uniform exponential model geometry


In [ ]:
import os 
#os.environ["JAX_PLATFORM_NAME"] = "cpu"
import jax 
jax.config.update("jax_enable_x64", True)
import jax_sbgeom
import jax.numpy as jnp
%load_ext autoreload
%autoreload 2
import numpy as onp
import matplotlib.pyplot as plt
import pyvista as pv
from jax_sbgeom.jax_utils.utils import _mesh_to_pyvista_mesh

##### Setting up geometry

Selecting a particular coil and plasma set:

In [ ]:
stell_i = -1
vmec_file = ["/home/tbogaarts/stellarator_paper/base_data/vmecs/helias3_vmec.nc4",     "/home/tbogaarts/stellarator_paper/base_data/vmecs/helias4_vmec.nc4", "/home/tbogaarts/stellarator_paper/base_data/vmecs/helias5_vmec.nc4",     "/home/tbogaarts/stellarator_paper/base_data/vmecs/squid_vmec.nc4"][stell_i]
coil_file = ["/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS3_coils_all.h5", "/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS4_coils_all.h5", "/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS5_coils_all.h5", "/home/tbogaarts/stellarator_paper/base_data/vmecs/squid_coilset.h5"][stell_i]

We use a FluxSurfaceConstantPhi, as this has the property that $\phi_{in} = \phi_{out}$ even beyond the LCFS (as required by FFTs). Furthermore, this creates a surface exactly the same as the original normal vector for constant $d$.

In [ ]:
from jax_sbgeom.flux_surfaces import FluxSurfaceNormalExtendedConstantPhi, ToroidalExtent, FluxSurfaceFourierExtended, FluxSurfaceNormalExtendedNoPhi
from jax_sbgeom.coils         import CoilSet, DiscreteCoil, convert_to_fourier_coilset, RotationMinimizedFrame, FiniteSizeCoilSet
flux_surface = FluxSurfaceNormalExtendedNoPhi.from_hdf5(vmec_file)
def _get_discrete_coils(coil_file):
    import h5py
    with h5py.File(coil_file, 'r') as f:
        coil_data = jnp.array(f['Dataset1'])
    return CoilSet.from_list([DiscreteCoil.from_positions(coil_data[i]) for i in range(coil_data.shape[0])])
coilset         = _get_discrete_coils(coil_file)
fourier_coilset = convert_to_fourier_coilset(coilset)
fs_coilset      = FiniteSizeCoilSet.from_coilset(jax_sbgeom.coils.coilset.filter_coilset_phi(fourier_coilset, 0.0, 2 * jnp.pi / 2.0 / flux_surface.settings.nfp), RotationMinimizedFrame, 100)
mesh_coils      = jax_sbgeom.coils.mesh_coilset_surface(fs_coilset, 500, 0.2, 0.2)


In [ ]:
# fw_distance         = 0.2 
# n_theta_blanket     = 55
# n_phi_blanket       = 65
# resolutions_blanket = [10, 1,1,6,4,3,4,3]  # Number of radial elements in each blanket layer
# thicknesses         = [0.002, 0.025, 0.5, 0.385, 0.06, 0.2, 0.06]  # Thickness of each blanket layer
# layers_jax          = jnp.concatenate([jnp.array([0.0]),jnp.cumsum(jnp.array(thicknesses))]) + fw_distance

In [ ]:
alpha   = 2.897
#alpha   = 1.6
beta    = alpha
gamma   = 3.38
delta   = 6.2
epsilon = 8.06

epsilon = 6.0

cws_kwargs = {
            'ncoil_splines'       : 130, # rather arbitrary
            'ntor_splines'        : 205,
            'nphi_rt'             : 120,
            'ntheta_rt'           : 120,
            'smoothing'           : 1.0
        }

zeta    = delta
blanket_dict = {
    "SOL" : {
        "decay" : 0.0,
        "d"     : 0.2,
        'resolution' : 1
    },
    "FirstWall" : {
        "decay" : alpha,        
        "d"     : 0.027,
        'resolution' : 1
    },
    "BreedingZone" : {
        "decay" : beta, 
        'resolution' : 5               
    },
    "BackSupportingStructure" : {
        "decay" : gamma,       
        "d"     : 0.385 ,
        'resolution' : 3
    },
    "VVWallFront" : {
        "decay" : delta,        
        "d"     : 0.1,
        'resolution' : 3
    },
    "VVShield" : {
        "decay" : epsilon,        
        'resolution' : 5
    },
    "VVWallBack" : {
        "decay" : zeta,    
        "d"     : 0.1,
        'resolution' : 5
    },
    "Void" : {
        'decay'      : 0.0,
        'resolution' : 7        
    }
}

exp_model_kwargs = {
            'phi_0'                   : 8e18,
            'phi_e'                   : 1e13,
            'maximum_dof_1_size'      : 1.0,
            'minimum_dof_2_size'      : 0.05,
            'smooth'                  : True,
            'smooth_sigma'            : 4,
            'dof_name_1'              : "BreedingZone",
            'dof_name_2'              : "VVShield"
        }
coil_thickness = 0.5

optimized_cws   = jax_sbgeom.coils.coil_winding_surface.create_optimized_coil_winding_surface(fourier_coilset, cws_kwargs['ncoil_splines'], cws_kwargs['ntor_splines'])
theta_d, phi_d, d_cws = jax_sbgeom.flux_surfaces.flux_surfaces_utilities.generate_thickness_matrix(flux_surface, optimized_cws, cws_kwargs['ntheta_rt'],cws_kwargs['nphi_rt'])
from scipy.ndimage import gaussian_filter
d_cws_smoothed = gaussian_filter(d_cws, sigma = cws_kwargs['smoothing'])

In [ ]:

def _find_bz_shield(blanket_dict, available_distance, phi_0, phi_e,  maximum_dof_1_size : float, minimum_dof_2_size : float, smooth = True, smooth_sigma = 4, dof_name_1 : str = "BreedingZone", dof_name_2 : str = "VVShield", void_key = "Void"):
    
    attenuation = onp.log10(phi_e / phi_0)    

    attenuation_without_bz_shield = sum([blanket_dict[layer]['decay'] * blanket_dict[layer]['d'] for layer in blanket_dict if (layer != dof_name_1 and layer != dof_name_2 and layer != void_key)])

    available_distance_for_bz_shield = available_distance - sum([blanket_dict[layer]['d'] for layer in blanket_dict if layer != dof_name_1 and layer != dof_name_2 and layer != void_key])
    
    required_attenuation_bz_shield = attenuation + attenuation_without_bz_shield
        
    available_bz = (required_attenuation_bz_shield + blanket_dict[dof_name_2]['decay'] * available_distance_for_bz_shield) / (blanket_dict[dof_name_2]['decay'] - blanket_dict[dof_name_1]['decay'])
    # if available_bz is greater than the allowable distance, it means that the shield needs to be 'negative' to satisfy the constraint. this just means that even if the 
    # entire volume is filled by the breeding zone, it is still below the constraint.

    # Now, the actual bz thickness is set to min(available_bz, maximum_dof_1_size) (we cap the bz thickness to 1m for practical reasons)    
    masked_bz =onp.minimum(available_bz, maximum_dof_1_size)# np.where(available_bz > maximum_dof_1_size, maximum_dof_1_size, available_bz)
    if smooth:        
        masked_bz = onp.minimum(gaussian_filter(masked_bz, sigma = smooth_sigma), masked_bz) # we can only smooth the minimum (otherwise we might violate the distance constraint)   
        
    required_shield_for_constraint = (required_attenuation_bz_shield + blanket_dict[dof_name_1]['decay'] * masked_bz) / ( -blanket_dict[dof_name_2]['decay'])        
    
    masked_shield =  onp.maximum(required_shield_for_constraint, minimum_dof_2_size)#np.where(required_shield_for_constraint < 0.05, 0.05,   required_shield_for_constraint)    

    blanket_keys_no_void = [key for key in blanket_dict.keys() if key != void_key]

    if onp.any(masked_shield + masked_bz > available_distance_for_bz_shield + 0.1 * blanket_dict[blanket_keys_no_void[-1]]["d"]):        
        print("Warning: required shield + bz exceeds available distance by:")
        print(onp.max(masked_shield + masked_bz - available_distance_for_bz_shield))
    
    return masked_bz, masked_shield

d_bz, d_shield = _find_bz_shield(blanket_dict, d_cws_smoothed - 0.5 * coil_thickness, **exp_model_kwargs)
blanket_dict_copy = blanket_dict.copy()
blanket_dict_copy['BreedingZone']['d'] = d_bz
blanket_dict_copy['VVShield']['d']     = d_shield
d_values                               = [blanket_dict_copy[i]['d'] for i in blanket_dict_copy.keys() if i!='Void' ]
total_d_matrices                       = [jnp.full_like(d_bz, d) if isinstance(d,float) else d for d in d_values]
cumsum_d_matrices                      = jnp.cumsum(jnp.array(total_d_matrices), axis=0) # Thickness matrices for all layers
void_d_matrix                          = d_cws_smoothed -  cumsum_d_matrices[-1]

In [ ]:
f_conversion = jax.jit(jax.vmap(jax_sbgeom.flux_surfaces.convert_to_vmec.create_fourier_surface_extension_interp_equal_arclength, in_axes=(None, 0, None, None, None)), static_argnums=(2,3,4))


In [ ]:
fourier_ext_kwargs = {
            'nu_sample' : 120, 
            'nv_sample' : 104,
            'mpol': 20,
            'ntor': 30,
            'ntheta_arclength' : 300
        }         
Rmnc_t, Zmns_t, mpol_t, ntor_t, nfp_t = f_conversion(flux_surface, cumsum_d_matrices, fourier_ext_kwargs['nu_sample'], fourier_ext_kwargs['nv_sample'], fourier_ext_kwargs['ntheta_arclength'])

In [ ]:
Rmnc_new = jax_sbgeom.flux_surfaces.convert_to_vmec._convert_to_different_ntor_mpol(Rmnc_t,fourier_ext_kwargs['mpol'], fourier_ext_kwargs['ntor'], int(mpol_t[0]), int(ntor_t[0]))
Zmns_new = jax_sbgeom.flux_surfaces.convert_to_vmec._convert_to_different_ntor_mpol(Zmns_t,fourier_ext_kwargs['mpol'], fourier_ext_kwargs['ntor'], int(mpol_t[0]), int(ntor_t[0]))

flux_surface_extension = jax_sbgeom.flux_surfaces.convert_to_vmec._create_fluxsurface_from_rmnc_zmns(Rmnc_new, Zmns_new, fourier_ext_kwargs['mpol'], fourier_ext_kwargs['ntor'], nfp_t[0], FluxSurfaceNormalExtendedNoPhi)
fs_new   = FluxSurfaceFourierExtended.from_flux_surface_and_extension(flux_surface, flux_surface_extension)

In [ ]:
d_matrix_from_edge = jax_sbgeom.flux_surfaces.flux_surfaces_utilities.generate_thickness_matrix(flux_surface_extension, optimized_cws, cws_kwargs['ntheta_rt'],cws_kwargs['nphi_rt'])

In [ ]:
d_matrix_from_edge_smoothed = gaussian_filter(d_matrix_from_edge[2], sigma = cws_kwargs['smoothing'])
plt.pcolormesh(d_matrix_from_edge_smoothed)
plt.colorbar()

In [ ]:
Rmnc_e, Zmns_e, mpol_e, ntor_e, nfp_e = jax_sbgeom.flux_surfaces.convert_to_vmec.create_fourier_surface_extension_interp(flux_surface_extension, d_matrix_from_edge_smoothed, fourier_ext_kwargs['nu_sample'], fourier_ext_kwargs['nv_sample'])

Rmnc_void = jax_sbgeom.flux_surfaces.convert_to_vmec._convert_to_different_ntor_mpol(Rmnc_e,fourier_ext_kwargs['mpol'], fourier_ext_kwargs['ntor'], int(mpol_t[0]), int(ntor_t[0]))
Zmns_void = jax_sbgeom.flux_surfaces.convert_to_vmec._convert_to_different_ntor_mpol(Zmns_e,fourier_ext_kwargs['mpol'], fourier_ext_kwargs['ntor'], int(mpol_t[0]), int(ntor_t[0]))

In [ ]:
Rmnc_total = jnp.concatenate([Rmnc_new, Rmnc_void[jnp.newaxis, :]], axis=0)
Zmns_total = jnp.concatenate([Zmns_new, Zmns_void[jnp.newaxis, :]], axis=0)

flux_surface_extension_total = jax_sbgeom.flux_surfaces.convert_to_vmec._create_fluxsurface_from_rmnc_zmns(Rmnc_total, Zmns_total, fourier_ext_kwargs['mpol'], fourier_ext_kwargs['ntor'], nfp_t[0])

fs_total = FluxSurfaceFourierExtended.from_flux_surface_and_extension(flux_surface, flux_surface_extension_total)

In [ ]:
Rmnc_total.shape

In [ ]:
n_surf_ext = fs_total.extension_flux_surface.data.Rmnc.shape[0]

ntheta = 55 
nphi   = 62

In [ ]:
surfaces = [] 
for i in range(n_surf_ext  + 1):
    surfaces.append(jax_sbgeom.flux_surfaces.mesh_surface(fs_total, 1.0 + i, ToroidalExtent.half_module(fs_new), ntheta, nphi))

In [ ]:
plotter = pv.Plotter()
colors = ['red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'violet', 'gray', 'white', 'cyan', 'magenta']
for s, i in enumerate(surfaces):
    mesh = _mesh_to_pyvista_mesh(*i)
    plotter.add_mesh(mesh, color = colors[s])

plotter.show()

In [ ]:
def plot_half_module_poloidal(no_phi_slices : int, fs_extended_base, key_list, ns = 160, plot_lines = True, base_plot_size = 5.0, width = None):

    from math import ceil
    from matplotlib.collections import LineCollection

    phi_slices = jnp.linspace(0, 2 * jnp.pi / fs_extended_base.settings.nfp/2.0, no_phi_slices, endpoint=True)
    colors = ['blue', 'green', 'yellow', 'orange', 'purple', 'cyan', 'black', 'lightgrey', 'brown', 'pink']

    no_surfaces= fs_extended_base.extension_flux_surface.data.Rmnc.shape[0]
    
    positions = [[fs_extended_base.cartesian_position(1.0+i, jnp.linspace(0, 2 * jnp.pi, ns), phi) for i in range(no_surfaces + 1)] for phi in phi_slices]
    


    pos_rs = jnp.array(positions).reshape(-1,3)
    
    pos_rs_r = jnp.sqrt(pos_rs[:,0]**2 + pos_rs[:,1]**2)
    pos_rs_z = pos_rs[:,2]
    xlims = [pos_rs_r.min(), pos_rs_r.max()]
    ylims = [pos_rs_z.min(), pos_rs_z.max()]

    ylims = jnp.stack([jnp.array(xlims), jnp.array(ylims)], axis=-1)

    margin = 0.05
    ymin = ylims[0] - margin * (ylims[1] - ylims[0])
    ymax = ylims[1] + margin * (ylims[1] - ylims[0])

    nrow = 2
    ncol = ceil(len(phi_slices)/2.0)

    if width is not None:
        base_plot_size = width / ncol

    fig, axs = plt.subplots(nrow, ncol, figsize=(base_plot_size * ncol, base_plot_size * nrow))
    for ax in axs.flat:
        ax.set_aspect('equal')    
        ax.set_xlim(ymin[0], ymax[0])
        ax.set_ylim(ymin[1], ymax[1])



    for idx, phi in enumerate(phi_slices):
        ax = axs.flat[idx]
        for i in range(no_surfaces):  # Fill between each pair of surfaces
        
            # Concatenate the boundary coordinates to form a closed polygon between two curves
            x = jnp.concatenate([positions[idx][i][:,0], positions[idx][i+1][::-1,0]])
            y = jnp.concatenate([positions[idx][i][:,1], positions[idx][i+1][::-1,1]])
            z = jnp.concatenate([positions[idx][i][:,2], positions[idx][i+1][::-1,2]])

            R = jnp.sqrt(x**2 + y**2)
            ax.fill(R, z, color=colors[i % len(colors)], alpha=1.0, label=key_list[i] if i < len(key_list) else "Outer Surface")

     
                    
        ax.set_aspect('equal')
        ax.set_title(f"$\\phi = {phi:.2f}$ rad")
        ax.set_xlim(ymin[0], ymax[0])
        ax.set_ylim(ymin[1], ymax[1])
        
        if idx == 0:
            ax.legend()
        if idx % ncol ==0:
            ax.set_ylabel('Z [m]')
        if idx  >= ncol * (nrow-1):
            ax.set_xlabel('R [m]')
    plt.tight_layout()
    return fig, axs
labels = [
        "Scrape-off-layer", "First Wall", "Breeding Zone", "Back Supporting Structure", "Vacuum Vessel Wall Front", "Shield", "Vacuum Vessel Wall Back", "Void"
    ]
fig, axs = plot_half_module_poloidal(5, fs_total, labels)

axs.flat[-1].axis('off')
axs.flat[0].get_legend().remove()
axs.flat[-1].legend(*axs.flat[0].get_legend_handles_labels(), framealpha = 0.0)

axs.flat[0].set_title("$\\phi = 0$ rad")
axs.flat[1].set_title("$\\phi = \\pi /16 $ rad")
axs.flat[2].set_title("$\\phi = \\pi /8 $ rad")
axs.flat[3].set_title("$\\phi = 3\\pi /16 $ rad")
axs.flat[-2].set_title("$\\phi = \\pi/4$ rad")

axs.flat[0].set_xticklabels([])
axs.flat[1].set_xticklabels([])
axs.flat[1].set_yticklabels([])
axs.flat[2].set_xticklabels([])
axs.flat[2].set_yticklabels([])
axs.flat[4].set_yticklabels([])


In [ ]:
from jax_sbgeom.interfaces import blanket_creation as bc

In [ ]:
d_tuple = tuple([cumsum_d_matrices[i] for i in range(cumsum_d_matrices.shape[0])]+ [d_cws_smoothed])

In [ ]:
resolutions = tuple([10] + [blanket_dict_copy[i]['resolution'] for i in blanket_dict_copy.keys()  if i != 'SOL'])

In [ ]:

ntheta = 55 
nphi   = 62
layered_squid = bc.LayeredDiscreteBlanket(d_layers=d_tuple, n_theta= ntheta, n_phi=nphi, resolution_layers = resolutions, toroidal_extent= ToroidalExtent.half_module(fs_total))

In [ ]:
tetrahedral_mesh = bc.mesh_tetrahedral_blanket(fs_total, layered_squid, 1)

In [ ]:

#n_layers = domain_settings.no_radial_elements 
total_array_layers = jnp.zeros(tetrahedral_mesh[1].shape[0], dtype=int)

actual_layers = [0 for i in range(resolutions[0])] + sum([[i+1 for b in range(resolutions[i + 1])] for i in range(len(resolutions) - 1)], start=[])

for i in range(layered_squid.n_discrete_layers):
    total_array_layers = total_array_layers.at[layered_squid.layer_slice(i)].set(actual_layers[i])

In [ ]:
plotter = pv.Plotter()
plotter.add_mesh(_mesh_to_pyvista_mesh(*tetrahedral_mesh), scalars = total_array_layers, opacity=1.0, show_edges=True, cmap = 'tab10')
plotter.add_mesh(_mesh_to_pyvista_mesh(*mesh_coils), color = 'white', opacity=1.0, show_edges=False)
plotter.show()